    Script for testing if Emotibit is recieveing artificial pulses generated from LSL.
        The following is tested for data colected with different time interval marker streams
        One stream with intervals of 1500ms
        Another stream of 10000ms (10s) intervals.
    

In [364]:
# %matplotlib inline
import numpy as np
import matplotlib.pyplot   as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path
# %matplotlib qt
sys.path.append(Path(r"C:\Users\ArturoV\Desktop\projects\EmotiBit\emotiBitData\LSL"))

In [365]:
%matplotlib qt


file1=r'..\emotiBitData\LSL\2023-01-17_18-09-42-832110\2023-01-17_18-09-42-832110_LM.csv'  
file2=r'..\emotiBitData\LSL\2023-01-17_19-00-10-012586\2023-01-17_19-00-10-012586_LM.csv'
file3=r'..\emotiBitData\LSL\2023-01-17_18-09-42-832110\2023-01-17_18-09-42-832110_AX.csv'
file4=r'..\emotiBitData\LSL\2023-01-17_19-00-10-012586\2023-01-17_19-00-10-012586_AX.csv'

data1=pd.read_csv(file1,delimiter=",")
data2=pd.read_csv(file2,delimiter=",")
HR_05s=pd.read_csv(file3,delimiter=",")
HR_1_5s=pd.read_csv(file4,delimiter=",")

Lets plot some figures

In [366]:
sns.set_theme()
fig,ax = plt.subplots(1,1)
ax.plot(HR_1_5s['LocalTimestamp'],HR_1_5s['AX'],'-',color='red')
for vline in data2['LocalTimestamp']:
    plt.axvline(vline, color = 'b', label = 'axvline - full height')
ax.set_title('1500ms interval data using LocalTimestamp')

fig2,ax2 = plt.subplots(1,1)
ax2.plot(HR_05s['LocalTimestamp'],HR_05s['AX'],'o-',color='red')
for vline in data1['LocalTimestamp']:
    plt.axvline(vline, color = 'b', label = 'axvline - full height')
ax2.set_title('500ms interval data using LocalTimestamp')

fig3,ax4 = plt.subplots(1,1)
ax4.plot(HR_05s['LslMarkerSourceTimestamp'],HR_05s['AX'],'o-',color='red')
for vline in data1['LslMarkerSourceTimestamp']:
    plt.axvline(vline, color = 'b', label = 'axvline - full height')
ax4.set_title('500ms interval data using LslMarkerSourceTimestamp')

Text(0.5, 1.0, '500ms interval data using LslMarkerSourceTimestamp')

Check the precision of the intervals sent by LSL and recieved by EmotiBit.
The precision needs to be improved. Can it be improved??
Possible Improvements-Investigations:

1. Better code for signal generation (Rhe signal generated and streamed to the LSL is very reliable--see xdf.reader.py)
2. Explore and play with the configuration of the LSL data parser.https://github.com/EmotiBit/EmotiBit_Docs/blob/master/Working_with_emotibit_data.md
3. Take a look at the LSL notes of emotibit and the ofxLSL library. https://github.com/EmotiBit/EmotiBit_Docs/blob/master/Learn_more_about_emotibit.md

It is important to take note that the problem comes directly from how emotibit reads the data from LSL. The signal present in LSL is completely precise. Maibe it is inevitable and we must count with certain delays windows. Take a look at bibliography.

In [367]:
times1=np.array(data1['LocalTimestamp'])
times2=np.array(data2['LocalTimestamp'])


In [368]:
difftime1=np.diff(times1)
difftime2=np.diff(times2)
mean_time1=np.mean(difftime1)
std_time1=np.std(difftime1)
mean_time2=np.mean(difftime2)
std_time2=np.std(difftime2)

difftime1_diff=np.absolute(difftime1-0.500)
difftime2_diff=np.absolute(difftime2-1.500)



In [369]:
nbins=15
fig1,ax1=plt.subplots(1,2,)
sns.histplot(data=difftime1,ax= ax1[0],bins=nbins)
sns.histplot(data=difftime2,ax= ax1[1],bins=nbins)
fig1.set_figheight(10)
fig1.set_figwidth(15)
ax1[0].set_title('500ms abs marker latency  mean = {}({}std)'.format(np.round(mean_time1,3),np.round(std_time1,3)))
ax1[1].set_title('1500ms abs marker latency LocalTimestamp mean = {}({}std)'.format(np.round(mean_time2,3),np.round(std_time2,3)))

ax1[0].axvline(mean_time1, color='black',linestyle='dashed')
ax1[0].axvline(mean_time1-std_time1, color='red',linestyle='dashed')
ax1[0].axvline(mean_time1+std_time1, color='red',linestyle='dashed')

ax1[1].axvline(mean_time2, color='black',linestyle='dashed')
ax1[1].axvline(mean_time2+std_time1, color='red',linestyle='dashed')
ax1[1].axvline(mean_time2-std_time1, color='red',linestyle='dashed')

ax1[0].xaxis.set_label_text('LocalTimestamp \n (s)')
ax1[1].xaxis.set_label_text('LocalTimestamp \n (s)')

ax1[0].xaxis.set_ticks(np.round(np.linspace(min(difftime1),max(difftime1),nbins),1))
ax1[0].xaxis.set_tick_params(reset=True,bottom=True,top=False)
ax1[1].xaxis.set_ticks(np.round(np.linspace(min(difftime2),max(difftime2),nbins),1))
ax1[1].xaxis.set_tick_params(reset=True,bottom=True,top=False)

In [370]:
fig1,ax1=plt.subplots(1,2,)
sns.histplot(data=difftime1_diff,ax= ax1[0],bins=5,binwidth=0.03,stat='probability')
sns.histplot(data=difftime2_diff,ax= ax1[1],bins=5,binwidth=0.03,stat='probability')
fig1.set_figheight(10)
fig1.set_figwidth(15)
ax1[0].set_title('500ms abslolute marker latency')
ax1[1].set_title('1500ms absolute marker latecy')
ax1[0].xaxis.set_ticks(np.arange(0,max(difftime1_diff),0.03))
ax1[0].xaxis.set_tick_params(reset=True,bottom=True,top=False)
ax1[1].xaxis.set_ticks(np.arange(0,max(difftime2_diff),0.03))
ax1[1].xaxis.set_tick_params(reset=True,bottom=True,top=False)
ax1[0].xaxis.set_label_text('LocalTimestamp \n (s)')
ax1[1].xaxis.set_label_text('LocalTimestamp \n (s)')


Text(0.5, 0, 'LocalTimestamp \n (s)')

Check signal latency streamed in LSL. 
The latency of the digital signal streamed into LSL is almost perfect.
In this example the signal was programmed with a latency of 500ms.

In [371]:
import pyxdf
data, header = pyxdf.load_xdf('../emotiBitData/LSL/2023-01-17_18-09-42-832110/sub-P001_ses-S001_task-Default_run-001_eeg.xdf')
for stream in data:
    y = stream['time_series']
    fig,ax = plt.subplots(1,1)

    if isinstance(y, list):
        # list of strings, draw one vertical line for each marker
        for timestamp, marker in zip(stream['time_stamps'], y):
            plt.axvline(x=timestamp)
            print(f'Marker "{marker[0]}" @ {timestamp:.2f}s')
    elif isinstance(y, np.ndarray):
        # numeric data, draw as lines
        plt.plot(stream['time_stamps'], y)
    else:
        raise RuntimeError('Unknown stream format')

# %%     Check signal latency
lsl_marker_signal_latency=np.diff(stream['time_stamps'])-0.5

fig6, ax6 = plt.subplots()
sns.histplot(data=signal_latency,bins=10)
ax.set_xlabel('Latency')
ax.set_ylabel('Frecuency')
ax.set_title('Signal latency')
ax.text(np.mean(lsl_marker_signal_latency),2,'Max latency {}'.format(max(lsl_marker_signal_latency)),fontsize=10)
ax.text(np.mean(lsl_marker_signal_latency),1,'Mean latency {}'.format(np.mean(lsl_marker_signal_latency)),fontsize=10)

lsl_marker_signal_latency_pd=pd.DataFrame(lsl_marker_signal_latency)
lsl_marker_signal_latency_pd.describe()
# print(('500ms abs lsl_marker_signal_latency  mean = {}({}std)'.format(round(np.mean(lsl_marker_signal_latency),3),round(np.std(lsl_marker_signal_latency),3))))

Marker "sync_event" @ 637277.65s
Marker "sync_event" @ 637278.15s
Marker "sync_event" @ 637278.65s
Marker "sync_event" @ 637279.15s
Marker "sync_event" @ 637279.65s
Marker "sync_event" @ 637280.15s
Marker "sync_event" @ 637280.65s
Marker "sync_event" @ 637281.15s
Marker "sync_event" @ 637281.65s
Marker "sync_event" @ 637282.15s
Marker "sync_event" @ 637282.65s
Marker "sync_event" @ 637283.15s
Marker "sync_event" @ 637283.65s
Marker "sync_event" @ 637284.15s
Marker "sync_event" @ 637284.65s
Marker "sync_event" @ 637285.15s
Marker "sync_event" @ 637285.65s
Marker "sync_event" @ 637286.15s
Marker "sync_event" @ 637286.65s
Marker "sync_event" @ 637287.15s
Marker "sync_event" @ 637287.66s
Marker "sync_event" @ 637288.16s
Marker "sync_event" @ 637288.66s
Marker "sync_event" @ 637289.16s
Marker "sync_event" @ 637289.66s
Marker "sync_event" @ 637290.16s
Marker "sync_event" @ 637290.66s
Marker "sync_event" @ 637291.16s
Marker "sync_event" @ 637291.66s
Marker "sync_event" @ 637292.16s
Marker "sy

,0
count,1245.000000
mean,0.001507
std,0.003480
min,-0.000174
25%,0.000234
50%,0.000521
75%,0.000874
max,0.015764


Are the discrepancies appearing at certain timestamps?


In [372]:
sns.jointplot(x=times1[:-1],y=difftime1,ax=ax8[0])


NameError: name 'ax8' is not defined